In [1]:
import numpy as np
import torch
import torch.nn as nn
import snntorch as snn
from snntorch import utils
from snntorch import surrogate
import torch.nn.functional as F
from snntorch import functional as SF
import brevitas.nn as qnn 
from tqdm import tqdm
from pathlib import Path

import sys
sys.path.append('../src')
from networks_debug import *
from dataloader import WisdmDatasetParser, WisdmDataset
from torch.utils.data import  DataLoader
from assistant import Assistant
from stats import LearningStats
from utils import *

In [2]:

#device = torch.device("cuda") if torch.cuda.is_available() else torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
device = 'cuda'
print(f'Using device {device}')

Using device cuda


In [3]:
batch_size = 512
slope = 10
# network parameters
num_inputs = 6 
num_steps = 40
net_hidden_1 = 180
net_hidden_2 = 400
net_hidden_3 = 128
num_outputs = 7
pop_outputs = num_outputs * 10
num_epochs = 200
vth_in = 0.4
vth_out = 0.5
vth_recurrent = 1.0
vth_enc_value =  0.4
vth_std =  65 
beta_in = 0.5
beta_recurrent = 0.5
beta_back = 0.7
beta_out = 0.2
encoder_dim = 25
beta_std = 55
lr = 0.002
drop_recurrent =0.15
drop_back = 0.35
drop_out = 0.35
# spiking neuron parameters
beta = 0.8  # neuron decay rate


In [4]:
DATASET_NAME = 'data_watch_subset_0_40.npz'
DATASET_SUBSET = '7BC'
PATIENCE = 12
TRAIN_FOLDER_NAME = 'Trained'
NUM_WORKERS = 8
NET_OUTPUT_DIM = 7
NET_INPUT_DIM = 6
NUM_EPOCHS = 200
SEARCH_SPACE_SHUFFLE = 200

In [5]:
trained_folder = TRAIN_FOLDER_NAME
os.makedirs(trained_folder, exist_ok=True)
dataset = WisdmDatasetParser(f'{Path.home()}/snntorch_network/data/{DATASET_NAME}', norm=None, class_sublset=DATASET_SUBSET)
train_set = dataset.get_training_set()
val_set = dataset.get_validation_set()


data, label = train_set
print(data.shape)

train_dataset = WisdmDataset(train_set)
val_dataset = WisdmDataset(val_set)

train_loader = DataLoader(dataset=train_dataset, batch_size=int(batch_size), shuffle=True, num_workers=NUM_WORKERS)
val_loader  = DataLoader(dataset= val_dataset, batch_size=int(batch_size), shuffle=True, num_workers=NUM_WORKERS)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f'Using device {device}')

grad = surrogate.fast_sigmoid(slope) #use slope for HPO

net_loss = regularization_loss(0.1, 0.03, 40)

net = QuantAhpcNetwork(NET_INPUT_DIM, int(net_hidden_1), int(net_hidden_2), NET_OUTPUT_DIM, grad,
                    vth_in=vth_in, vth_recurrent=vth_recurrent, vth_out=vth_out,
                    beta_in=beta_in, beta_recurrent=beta_recurrent, beta_back=beta_back, beta_out=beta_out,
                    encoder_dim=int(encoder_dim),
                    vth_enc_value=vth_enc_value, vth_std=vth_std, beta_std=beta_std,
                    drop_recurrent=drop_recurrent, drop_back=drop_back, drop_out=drop_out,
                    time_dim=2, layer_loss=net_loss).to(device)

optimizer = torch.optim.Adam(net.parameters(), lr=lr, betas=(0.9, 0.999))
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 
    T_max=4690, 
    eta_min=0, 
    last_epoch=-1
)

loss_fn = SF.loss.ce_count_loss()

stats = LearningStats()
assistant = Assistant(net, loss_fn, optimizer, stats, classifier=True, scheduler=scheduler, lam=1.0)

count = 0
for epoch in range(NUM_EPOCHS):
    labels = []
    outputs = []
    # if epoch % 20 == 0:
    #     assistant.reduce_lr()
    if count < PATIENCE:
        count = count+1
        tqdm_dataloader = tqdm(train_loader)
        for _, batch in enumerate(tqdm_dataloader): # training loop
            input, label = batch
            
            output = assistant.train(input, label)
            tqdm_dataloader.set_description(f'\r[Epoch {epoch:2d}/{NUM_EPOCHS}] Training: {stats.training}')

        tqdm_dataloader = tqdm(val_loader)
        for _, batch in enumerate(tqdm_dataloader): #eval loop
            input, label = batch
            output = assistant.test(input, label)
            tqdm_dataloader.set_description(f'\r[Epoch {epoch:2d}/{NUM_EPOCHS}] Validation: {stats.testing}')
        
            if len(outputs) == 0:
                outputs = output.to('cpu').detach()
                labels = label.to('cpu').detach()
            else:
                outputs = torch.cat((outputs, output.to('cpu').detach()), dim=1)
                labels = torch.cat((labels, label.to('cpu').detach()))

        stats.update()

        if stats.testing.best_accuracy:
            count = 0
            _, predictions = outputs.sum(dim=0).max(1)
            gen_confusion_matrix(predictions,labels, f'./{trained_folder}/')
            net.save_to_npz(f'./{trained_folder}/network_best.npz')
            stats.save( f'./{trained_folder}/')

        del outputs
        del labels
        torch.cuda.empty_cache()
    else:
        print('Early stopping')
        break

net.load_from_npz(f'./{trained_folder}/network_best.npz')

tqdm_dataloader = tqdm(val_loader)
for _, batch in enumerate(tqdm_dataloader): #eval loop
    input, label = batch
    output = assistant.test(input, label)
    tqdm_dataloader.set_description(f'\r[Epoch {epoch:2d}/{NUM_EPOCHS}] Final Validation: {stats.testing}')


    

    



/home/franzhd/miniconda3/envs/snn_torch/lib/python3.10/site-packages/brevitas/nn/mixin/base.py:77: UserWarning: Keyword arguments are being passed but they not being used.
  warn('Keyword arguments are being passed but they not being used.')


(6,)
(6,)
ytrain shape (55404, 18)
yval shape (18468, 18)
ytest shape (18469, 18)
num classes train dataset: 7 occurrences of each class:[3066 3102 2981 3047 3095 2973 3213]
num classes eval dataset: 7 occurrences of each class:[ 968 1122  982  996  966 1007 1077]
num classes test dataset: 7 occurrences of each class:[1061 1046  989 1036  965  982 1054]
(21477, 6, 40)
Using device cuda


  0%|          | 0/42 [00:00<?, ?it/s]/home/franzhd/.local/lib/python3.10/site-packages/torch/_tensor.py:1394: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at ../c10/core/TensorImpl.h:1908.)
  return super().rename(names)
[Epoch  0/200] Training: loss =     1.88518                          accuracy = 0.26582 : 100%|██████████| 42/42 [00:26<00:00,  1.59it/s]
[Epoch  0/200] Validation: loss =     1.63769                          accuracy = 0.33577 : 100%|██████████| 14/14 [00:04<00:00,  3.23it/s]
[Epoch  1/200] Training: loss =     1.59454 (min =     1.88518)     accuracy = 0.41761 (max = 0.26582) : 100%|██████████| 42/42 [00:26<00:00,  1.60it/s]
[Epoch  1/200] Validation: loss =     1.52804 (min =     1.63769)     accuracy = 0.38915 (max = 0.33577) : 100%|██████████| 14/14 [00:04<00:00,  2.90it/s]
[Epoch  2/200] Training: l